#### numpy实现DNN梯度下降

In [22]:
# 以下是一个简单的用numpy实现的手写梯度下降的神经网络的例子，其中用到了反向传播算法来更新权重和偏差，实现了对二分类问题的分类：
import numpy as np

# sigmoid函数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# sigmoid函数的导数
def sigmoid_derivative(x):
    return x * (1 - x)

# 输入数据
X = np.array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
# 输出数据
y = np.array([[0, 0, 1, 1]]).T

# 随机初始化权重和偏差
np.random.seed(1)
weights = 2 * np.random.random((3, 1)) - 1
bias = 2 * np.random.random(1) - 1

# 设置学习率
learning_rate = 0.1

# 设置训练次数
num_epochs = 10000

# 训练
for epoch in range(num_epochs):
    # 前向传播
    inputs = np.dot(X, weights) + bias
    outputs = sigmoid(inputs)

    # 计算误差
    error = y - outputs

    # 计算梯度
    gradients = error * sigmoid_derivative(outputs)

    # 更新权重和偏差
    weights += learning_rate * np.dot(X.T, gradients)
    bias += learning_rate * np.sum(gradients)

# 打印最终的权重和偏差
print(weights)
print(bias)
print(y,outputs)

[[ 7.34445135]
 [-0.1168797 ]
 [-2.08005738]]
[-1.47562099]
[[0]
 [0]
 [1]
 [1]] [[0.02777032]
 [0.02478295]
 [0.97787599]
 [0.97520098]]


####  wide & deep 

In [54]:
# 分类
import tensorflow as tf
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load wine data
data = load_wine()
X = data.data
y = data.target

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the wide and deep parts of the model
input_wide = tf.keras.layers.Input(shape=(7,))   #shape=(X_train.shape[1],)
input_wide2 = tf.keras.layers.Dense(3)(input_wide)
output_wide = input_wide2

input_deep = tf.keras.layers.Input(shape=(6,))   #shape=(X_train.shape[1],)
hidden = tf.keras.layers.Dense(units=64, activation='relu')(input_deep)
output_deep = tf.keras.layers.Dense(units=3, activation='softmax')(hidden)


# Merge the wide and deep parts of the model
merged = tf.keras.layers.concatenate([output_wide, output_deep])
output = tf.keras.layers.Dense(units=3, activation='softmax')(merged)
merged_model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=output)

# Compile the model
merged_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
merged_model.fit([X_train[:,6:], X_train[:,:6]], y_train, epochs=10, batch_size=32)

# Evaluate the model on the test data
test_loss, test_acc = merged_model.evaluate([X_test[:,6:], X_test[:,:6]], y_test, verbose=0)
print("Test accuracy: {:.3f}".format(test_acc))


Epoch 1/10
5/5 [==============================] - 0s 1ms/step - loss: 1.3390 - accuracy: 0.1408
Epoch 2/10
5/5 [==============================] - 0s 892us/step - loss: 1.2995 - accuracy: 0.1690
Epoch 3/10
5/5 [==============================] - 0s 2ms/step - loss: 1.2611 - accuracy: 0.1972
Epoch 4/10
5/5 [==============================] - 0s 2ms/step - loss: 1.2248 - accuracy: 0.2535
Epoch 5/10
5/5 [==============================] - 0s 1ms/step - loss: 1.1895 - accuracy: 0.2817
Epoch 6/10
5/5 [==============================] - 0s 1ms/step - loss: 1.1557 - accuracy: 0.3239
Epoch 7/10
5/5 [==============================] - 0s 1ms/step - loss: 1.1235 - accuracy: 0.3803
Epoch 8/10
5/5 [==============================] - 0s 1ms/step - loss: 1.0914 - accuracy: 0.4085
Epoch 9/10
5/5 [==============================] - 0s 2ms/step - loss: 1.0605 - accuracy: 0.4507
Epoch 10/10
5/5 [==============================] - 0s 1ms/step - loss: 1.0308 - accuracy: 0.4507
Test accuracy: 0.444


In [62]:
# 回归
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the California housing dataset
data = fetch_california_housing()
X = data.data
y = data.target

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the input layers for the wide and deep parts of the model
input_wide = tf.keras.layers.Input(shape=(X_train.shape[1],))
input_deep = tf.keras.layers.Input(shape=(X_train.shape[1],))

# Define the wide part of the model
hidden_wide_1 = tf.keras.layers.Dense(units=32, activation='relu')(input_wide)
hidden_wide_2 = tf.keras.layers.Dense(units=32, activation='relu')(hidden_wide_1)
output_wide = tf.keras.layers.Dense(units=1, activation='linear')(hidden_wide_2)

# Define the deep part of the model
hidden_deep_1 = tf.keras.layers.Dense(units=64, activation='relu')(input_deep)
hidden_deep_2 = tf.keras.layers.Dense(units=64, activation='relu')(hidden_deep_1)
hidden_deep_3 = tf.keras.layers.Dense(units=64, activation='relu')(hidden_deep_2)
output_deep = tf.keras.layers.Dense(units=1, activation='linear')(hidden_deep_3)

# Merge the wide and deep parts of the model
merged = tf.keras.layers.concatenate([output_wide, output_deep])
output = tf.keras.layers.Dense(units=1, activation='linear')(merged)

# Create the final model
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Fit the model to the training data
model.fit([X_train, X_train], y_train, epochs=20, batch_size=32)
# Evaluate the model on the test data
test_loss, test_mae = model.evaluate([X_test, X_test], y_test)
print('Test loss:', test_loss)
print('Test MAE:', test_mae)



Epoch 1/20
516/516 [==============================] - 1s 962us/step - loss: 0.6624 - mean_absolute_error: 0.5602
Epoch 2/20
516/516 [==============================] - 0s 903us/step - loss: 0.3835 - mean_absolute_error: 0.4327
Epoch 3/20
516/516 [==============================] - 1s 1ms/step - loss: 0.3428 - mean_absolute_error: 0.4141
Epoch 4/20
516/516 [==============================] - 0s 873us/step - loss: 0.3297 - mean_absolute_error: 0.3999
Epoch 5/20
516/516 [==============================] - 0s 899us/step - loss: 0.3077 - mean_absolute_error: 0.3888
Epoch 6/20
516/516 [==============================] - 0s 886us/step - loss: 0.3036 - mean_absolute_error: 0.3845
Epoch 7/20
516/516 [==============================] - 0s 884us/step - loss: 0.2937 - mean_absolute_error: 0.3754
Epoch 8/20
516/516 [==============================] - 0s 886us/step - loss: 0.2862 - mean_absolute_error: 0.3707
Epoch 9/20
516/516 [==============================] - 0s 878us/step - loss: 0.2942 - mean_absolute

In [80]:
# import torch
# from torch import nn
import jieba
import numpy as np
import tensorflow as tf

raw_text = '越努力越幸运'
words = list(jieba.cut(raw_text))
word_to_ix = {i:word for i,word in enumerate(set(words))} #索引化

keys = word_to_ix.keys()
keys_list = list(keys) # [0, 1, 2]

embeds = tf.keras.layers.Embedding(4,3)
embeds(tf.Variable(keys_list))

# embeds = nn.Embedding(4,3)
# embeds(torch.LongTensor(keys_list))


<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 0.04220146, -0.03328271, -0.04742723],
       [ 0.04385703, -0.02046537, -0.0095141 ],
       [ 0.014242  ,  0.00292666,  0.00830215]], dtype=float32)>

In [19]:
#定义elo score 等级评分类
class EloScore:
    #初始积分
    ELO_RATING_DEFAULT = 1500

    #定义初始化方法
    def __init__(self,Sa,ratingA=ELO_RATING_DEFAULT,ratingB=ELO_RATING_DEFAULT):
        self.Sa = Sa
        self.ratingA = ratingA
        self.ratingB = ratingB
      
    #定义阈值 k值
    def computeK(self,rating):
        if rating >=2400:
            return 16
        elif rating >= 2100:
            return 24
        else:
            return 36

    #使用公式推算
    def computeScore(self,):
        Eb_S = 1 / (1+pow(10,(self.ratingA-self.ratingB)/400))  #B对A的胜概率
        Ea_S = 1 - Eb_S #A对B的胜概率
        return Ea_S,Eb_S
    
    def balanceK(self,):
        avg_K = (self.computeK(self.ratingA) + self.computeK(self.ratingB))/2
        return avg_K
    
    def main(self,):
        K = self.balanceK()
        Ea_S, Eb_S = self.computeScore()
        print(Ea_S,Eb_S)
        
        Sa, Sb = self.Sa , 1 - self.Sa    # 实际胜负 WIN = 1, LOSS = 0, TIE = 0.5
        
        Ra = K * (Sa - Ea_S)
        Rb = K * (Sb - Eb_S)
        return Ra,Rb
if __name__ == "__main__":

    eloscore = EloScore(0,1800,1500)
    print(eloscore.main())

0.8490204427886767 0.15097955721132328
(-30.56473594039236, 20.37649062692824)


In [15]:
36*(0.5-0.8490204427886767)

-12.564735940392362

In [9]:
pow(10,(300)/400)

5.623413251903491

In [11]:
1/6.623

0.15098897780462026

In [12]:
pow(10,(800)/400)

100.0

In [13]:
1/101

0.009900990099009901